# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 27 2022 12:43PM,249532,12,HH-37800,Hush Hush,Released
1,Oct 27 2022 12:43PM,249532,12,HH-37801,Hush Hush,Released
2,Oct 27 2022 12:43PM,249532,12,HH-37802,Hush Hush,Released
3,Oct 27 2022 12:43PM,249532,12,HH-37803,Hush Hush,Released
4,Oct 27 2022 12:43PM,249532,12,HH-37804,Hush Hush,Released
5,Oct 27 2022 12:43PM,249532,12,HH-37805,Hush Hush,Released
6,Oct 27 2022 12:43PM,249532,12,HH-37806,Hush Hush,Released
7,Oct 27 2022 12:30PM,249525,19,8686745,Eywa Pharma Inc.,Released
8,Oct 27 2022 12:30PM,249525,19,8691191,Eywa Pharma Inc.,Released
9,Oct 27 2022 12:24PM,249531,10,0086122705,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,249529,Released,3
29,249530,Released,39
30,249531,Executing,3
31,249531,Released,9
32,249532,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249528,NaN,NaN,4.0
249529,NaN,NaN,3.0
249530,NaN,NaN,39.0
249531,NaN,3.0,9.0
249532,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248349,0.0,0.0,1.0
249451,0.0,1.0,0.0
249472,11.0,15.0,15.0
249479,0.0,1.0,0.0
249485,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248349,0,0,1
249451,0,1,0
249472,11,15,15
249479,0,1,0
249485,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248349,0,0,1
1,249451,0,1,0
2,249472,11,15,15
3,249479,0,1,0
4,249485,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248349,,,1
1,249451,,1,
2,249472,11,15,15
3,249479,,1,
4,249485,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 27 2022 12:43PM,249532,12,Hush Hush
7,Oct 27 2022 12:30PM,249525,19,Eywa Pharma Inc.
9,Oct 27 2022 12:24PM,249531,10,ISDIN Corporation
21,Oct 27 2022 12:21PM,249528,10,"Methapharm, Inc."
25,Oct 27 2022 12:19PM,249530,15,"Person & Covey, Inc."
64,Oct 27 2022 12:16PM,249529,10,"Methapharm, Inc."
67,Oct 27 2022 12:15PM,249527,10,Emerginnova
70,Oct 27 2022 12:15PM,249526,21,"NBTY Global, Inc."
71,Oct 27 2022 12:14PM,249524,21,"NBTY Global, Inc."
72,Oct 27 2022 12:13PM,249523,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 27 2022 12:43PM,249532,12,Hush Hush,,,7
1,Oct 27 2022 12:30PM,249525,19,Eywa Pharma Inc.,,,2
2,Oct 27 2022 12:24PM,249531,10,ISDIN Corporation,,3,9
3,Oct 27 2022 12:21PM,249528,10,"Methapharm, Inc.",,,4
4,Oct 27 2022 12:19PM,249530,15,"Person & Covey, Inc.",,,39
5,Oct 27 2022 12:16PM,249529,10,"Methapharm, Inc.",,,3
6,Oct 27 2022 12:15PM,249527,10,Emerginnova,,,3
7,Oct 27 2022 12:15PM,249526,21,"NBTY Global, Inc.",,,1
8,Oct 27 2022 12:14PM,249524,21,"NBTY Global, Inc.",,,1
9,Oct 27 2022 12:13PM,249523,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 12:43PM,249532,12,Hush Hush,7,,
1,Oct 27 2022 12:30PM,249525,19,Eywa Pharma Inc.,2,,
2,Oct 27 2022 12:24PM,249531,10,ISDIN Corporation,9,3,
3,Oct 27 2022 12:21PM,249528,10,"Methapharm, Inc.",4,,
4,Oct 27 2022 12:19PM,249530,15,"Person & Covey, Inc.",39,,
5,Oct 27 2022 12:16PM,249529,10,"Methapharm, Inc.",3,,
6,Oct 27 2022 12:15PM,249527,10,Emerginnova,3,,
7,Oct 27 2022 12:15PM,249526,21,"NBTY Global, Inc.",1,,
8,Oct 27 2022 12:14PM,249524,21,"NBTY Global, Inc.",1,,
9,Oct 27 2022 12:13PM,249523,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 12:43PM,249532,12,Hush Hush,7,,
1,Oct 27 2022 12:30PM,249525,19,Eywa Pharma Inc.,2,,
2,Oct 27 2022 12:24PM,249531,10,ISDIN Corporation,9,3,
3,Oct 27 2022 12:21PM,249528,10,"Methapharm, Inc.",4,,
4,Oct 27 2022 12:19PM,249530,15,"Person & Covey, Inc.",39,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 12:43PM,249532,12,Hush Hush,7.0,NaN,NaN
1,Oct 27 2022 12:30PM,249525,19,Eywa Pharma Inc.,2.0,NaN,NaN
2,Oct 27 2022 12:24PM,249531,10,ISDIN Corporation,9.0,3.0,NaN
3,Oct 27 2022 12:21PM,249528,10,"Methapharm, Inc.",4.0,NaN,NaN
4,Oct 27 2022 12:19PM,249530,15,"Person & Covey, Inc.",39.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 12:43PM,249532,12,Hush Hush,7.0,0.0,0.0
1,Oct 27 2022 12:30PM,249525,19,Eywa Pharma Inc.,2.0,0.0,0.0
2,Oct 27 2022 12:24PM,249531,10,ISDIN Corporation,9.0,3.0,0.0
3,Oct 27 2022 12:21PM,249528,10,"Methapharm, Inc.",4.0,0.0,0.0
4,Oct 27 2022 12:19PM,249530,15,"Person & Covey, Inc.",39.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1497142,21.0,3.0,0.0
12,748539,8.0,1.0,0.0
15,1497054,87.0,15.0,12.0
16,998039,3.0,1.0,0.0
19,499034,3.0,0.0,0.0
20,498979,1.0,1.0,0.0
21,1495875,5.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1497142,21.0,3.0,0.0
1,12,748539,8.0,1.0,0.0
2,15,1497054,87.0,15.0,12.0
3,16,998039,3.0,1.0,0.0
4,19,499034,3.0,0.0,0.0
5,20,498979,1.0,1.0,0.0
6,21,1495875,5.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,21.0,3.0,0.0
1,12,8.0,1.0,0.0
2,15,87.0,15.0,12.0
3,16,3.0,1.0,0.0
4,19,3.0,0.0,0.0
5,20,1.0,1.0,0.0
6,21,5.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,21.0
1,12,Released,8.0
2,15,Released,87.0
3,16,Released,3.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,12.0,0.0,0.0,0.0,0.0
Executing,3.0,1.0,15.0,1.0,0.0,1.0,1.0
Released,21.0,8.0,87.0,3.0,3.0,1.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,12.0,0.0,0.0,0.0,0.0
1,Executing,3.0,1.0,15.0,1.0,0.0,1.0,1.0
2,Released,21.0,8.0,87.0,3.0,3.0,1.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,12.0,0.0,0.0,0.0,0.0
1,Executing,3.0,1.0,15.0,1.0,0.0,1.0,1.0
2,Released,21.0,8.0,87.0,3.0,3.0,1.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()